# Topic Modeling
In this exercise, we will do topic modeling with gensim. Use the [topics and transformations tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html) as a reference.

In [3]:
import os
from collections import defaultdict

import gensim
import nltk

For tokenizing words and stopword removal, download the NLTK punkt tokenizer and stopwords list.

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nevin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nevin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

First, we load the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included with gensim that contains 300 news articles of the Australian Broadcasting Corporation.

In [5]:
from gensim.test.utils import datapath
train_file = datapath('lee_background.cor')
articles_orig = open(train_file).read().splitlines()

Preprocess the text by lowercasing, removing stopwords, stemming, and removing rare words.

In [6]:
# define stopword list
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords | {'\"', '\'', '\'\'', '`', '``', '\'s'}

# initialize stemmer
stemmer = nltk.stem.PorterStemmer()

def preprocess(article):
    # tokenize
    article = nltk.word_tokenize(article)

    # lowercase all words
    article = [word.lower() for word in article]

    # remove stopwords
    article = [word for word in article if word not in stopwords]

    # optional: stem
    # article = [stemmer.stem(word) for word in article]
    return article

articles = [preprocess(article) for article in articles_orig]

# create the dictionary and corpus objects that gensim uses for topic modeling
dictionary = gensim.corpora.Dictionary(articles)

# remove words that occur in less than 2 documents, or more than 50% of documents
dictionary.filter_extremes(no_below=2, no_above=0.5)
temp = dictionary[0]  # load the dictionary by calling it once
corpus_bow = [dictionary.doc2bow(article) for article in articles]


Now we create a TF-IDF model and transform the corpus into TF-IDF vectors.

In [9]:
model_tfidf = gensim.models.TfidfModel(corpus_bow)
corpus_tfidf = model_tfidf[corpus_bow]

print('BOW:')
print(corpus_bow[0])

print('TF-IDF:')
print(corpus_tfidf[0])


BOW:
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 7), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 2), (49, 2), (50, 3), (51, 3), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 8), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 5), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 3), (99, 1), (100, 1), (101, 3), (102, 1), (103, 1), (104, 1), (105, 4), (106, 2), (107, 1), (108, 1), (109, 1), (110

Now we train an [LDA model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html) with 10 topics on the TF-IDF corpus. Save it to a variable `model_lda`.

In [13]:
model_lda = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=10)

model_lda.print_topics(5)

[(6,
  '0.002*"mr" + 0.002*"zealand" + 0.001*"arafat" + 0.001*"palestinian" + 0.001*"rafter" + 0.001*"people" + 0.001*"asked" + 0.001*"asylum" + 0.001*"israeli" + 0.001*"secret"'),
 (1,
  '0.003*"workers" + 0.002*"qantas" + 0.002*"best" + 0.002*"industrial" + 0.002*"us" + 0.001*"unions" + 0.001*"yallourn" + 0.001*"hewitt" + 0.001*"power" + 0.001*"trees"'),
 (2,
  '0.003*"palestinian" + 0.002*"israel" + 0.002*"hamas" + 0.002*"party" + 0.001*"arafat" + 0.001*"suicide" + 0.001*"minister" + 0.001*"israeli" + 0.001*"two" + 0.001*"city"'),
 (8,
  '0.002*"club" + 0.002*"commission" + 0.001*"afghanistan" + 0.001*"lockett" + 0.001*"us" + 0.001*"bin" + 0.001*"laden" + 0.001*"mr" + 0.001*"south" + 0.001*"year"'),
 (7,
  '0.002*"governor-general" + 0.002*"hollingworth" + 0.002*"test" + 0.002*"dr" + 0.001*"woomera" + 0.001*"centre" + 0.001*"republic" + 0.001*"child" + 0.001*"people" + 0.001*"oil"')]

Let's inspect the first 5 topics of our model.

In [11]:
topics = model_lda.print_topics(num_words=5)
print('LDA topics:')
for topic in topics:
    print(topic)

LDA topics:
(0, '0.003*"palestinian" + 0.002*"government" + 0.002*"afghanistan" + 0.002*"states" + 0.001*"-"')
(1, '0.002*"workers" + 0.002*"company" + 0.002*"pakistan" + 0.002*"friedli" + 0.002*"indian"')
(2, '0.002*"arafat" + 0.002*"palestinian" + 0.001*"israeli" + 0.001*"change" + 0.001*"staff"')
(3, '0.002*"reid" + 0.002*"man" + 0.002*"police" + 0.001*"hotel" + 0.001*"fire"')
(4, '0.003*"palestinian" + 0.002*"israeli" + 0.002*"arafat" + 0.001*"australia" + 0.001*"government"')
(5, '0.002*"test" + 0.002*"club" + 0.002*"afghanistan" + 0.002*"us" + 0.002*"south"')
(6, '0.002*"palestinian" + 0.002*"dr" + 0.002*"afghan" + 0.001*"australia" + 0.001*"agreement"')
(7, '0.001*"bill" + 0.001*"workers" + 0.001*"royal" + 0.001*"australia" + 0.001*"qantas"')
(8, '0.002*"yacht" + 0.002*"australia" + 0.002*"asic" + 0.002*"hobart" + 0.002*"sydney"')
(9, '0.002*"qantas" + 0.002*"workers" + 0.002*"centre" + 0.002*"woomera" + 0.002*"detainees"')


We see the 5 topics with the highest importance. For each topic, the 10 most important words are shown, together with their coefficient of "alignment" to the topic.

## Document Similarity
We now use our LDA model to compare the similarity of new documents (*queries*) to documents in our collection.

First, create an index of the news articles in our corpus. Use the `MatrixSimilarity` transformation as described in gensim's [similarity queries tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html).

In [15]:
index = gensim.similarities.MatrixSimilarity(model_lda[corpus_tfidf])


Now, write a function that takes a query string as input and returns the LDA representation for it. Make sure to apply the same preprocessing as we did to the documents.

In [16]:
def lda_representation(query):
    query = preprocess(query)
    query_bow = dictionary.doc2bow(query)
    query_tfidf = model_tfidf[query_bow]
    query_lda = model_lda[query_tfidf]
    return query_lda
    

Print the top 5 most similar documents, together with their similarities, using your index created above.

In [ ]:
query = 'Prime Minister of Australia'
query_lda = lda_representation(query)
sims = index[query_lda]

sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, sim in sims[:5]:
    print('Document:', i, 'Similarity:', sim)
    print(articles_orig[i])
    print()




Document: 207 Similarity: 0.99473554
Geoff Huegill has continued his record-breaking ways at the World Cup short course swimming in Melbourne, bettering the Australian record in the 100 metres butterfly. Huegill beat fellow Australian Michael Klim, backing up after last night setting a world record in the 50 metres butterfly. 

Document: 276 Similarity: 0.99407655
Defence Minister Robert Hill has confirmed Australian troops arrived in Afghanistan this morning. Senator Hill says it is an advance party and the rest of the troops will arrive within the next few days. He says Australian forces will operate with US troops in southern Afghanistan to fight the Taliban and Al Qaeda networks. Senator Hill says the operation could take several months. 

Document: 297 Similarity: 0.99315614
The Federal National Party has rejected a possible merger with the Liberals' at this stage, but it has not ruled out the option over the next three years. Liberal Party President Shane Stone is reported as say

Run your code again, now training an LDA model with 100 topics. Do you see a qualitative difference in the top-5 most similar documents?